# Hands-on!

Nessa prática, sugerimos alguns pequenos exemplos para você implementar sobre o Spark.

## Apriorando o Word Count Memória Postumas de Brás Cubas

Memórias Póstumas de Brás Cubas é um romance escrito por Machado de Assis, desenvolvido em princípio como folhetim, de março a dezembro de 1880, na Revista Brasileira, para, no ano seguinte, ser publicado como livro, pela então Tipografia Nacional.

A obra retrata a escravidão, as classes sociais, o cientificismo e o positivismo da época. Dada essas informações, será que conseguimos idenficar essas características pelas palavras mais utilizadas em sua obra?

Utilizando o dataset `Machado-de-Assis-Memorias-Postumas.txt`, elabore um `pipeline` utilizando `Estimators` e `Transformers` necessário do Spark para responder as perguntas abaixo. Não esqueça de utilizar `stopwords.pt` para remover as `stop words`!

- Quais as 10 palavras mais frequentes?
- Quais as 2-grams e 3-grams mais frequentes?


In [52]:
# Bibliotecas
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, NGram

In [53]:
livro = sc.textFile("Machado-de-Assis-Memorias-Postumas.txt")
text = ""
for line in livro.collect():
    text += " " + line

data = spark.createDataFrame([(0, text)], ["id", "text"])

### 10 Palavras Mais Frequentes

In [49]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
count = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10)

pipeline = Pipeline(stages=[tokenizer, remover, count])

In [50]:
model = pipeline.fit(data).transform(data)
model.select("features").show(truncate=False)

+---------------------------------------------------------------------------------------------+
|features                                                                                     |
+---------------------------------------------------------------------------------------------+
|(10,[0,1,2,3,4,5,6,7,8,9],[2116.0,2115.0,1885.0,1720.0,1089.0,981.0,745.0,652.0,650.0,581.0])|
+---------------------------------------------------------------------------------------------+



### n-Grams

In [60]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
ngram = NGram(inputCol="filtered", outputCol="ngrams", n=2)
count = CountVectorizer(inputCol="ngrams", outputCol="features", vocabSize=10)

pipeline = Pipeline(stages=[tokenizer, remover, ngram, count])

In [62]:
model = pipeline.fit(data).transform(data)
model.select("features").show(truncate=False)

+--------------------------------------------------------------------------------------+
|features                                                                              |
+--------------------------------------------------------------------------------------+
|(10,[0,1,2,3,4,5,6,7,8,9],[160.0,158.0,139.0,136.0,128.0,117.0,111.0,107.0,88.0,81.0])|
+--------------------------------------------------------------------------------------+



In [63]:
ngram = NGram(inputCol="filtered", outputCol="ngrams", n=3)

pipeline = Pipeline(stages=[tokenizer, remover, ngram, count])

In [64]:
model = pipeline.fit(data).transform(data)
model.select("features").show(truncate=False)

+------------------------------------------------------------------------------+
|features                                                                      |
+------------------------------------------------------------------------------+
|(10,[0,1,2,3,4,5,6,7,8,9],[23.0,22.0,20.0,17.0,15.0,15.0,14.0,14.0,11.0,11.0])|
+------------------------------------------------------------------------------+



## TF-IDF com CountVectorizer

No exemplo `TFIDF`, atualize a cell 15 para utilizar o Transformer `CountVectorizer`.

In [ ]:
countVect = CountVectorizer(inputCol="rawFeatures", outputCol="features")
countVectModel = countVect.fit(featurizedData)
rescaledData = countVectModel.transform(featurizedData)